In [1]:
# =========================
# Standard library
# =========================
import os
import ast

# =========================
# Third-party libraries
# =========================
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    roc_auc_score
)

# =========================
# RDKit
# =========================
from rdkit import Chem
from rdkit.Chem import (
    AllChem,
    MACCSkeys,
    rdMolDescriptors,
    RDKFingerprint,
    Descriptors
)
from rdkit.DataStructs import (
    TanimotoSimilarity,
    ExplicitBitVect
)

# =========================
# CDK
# =========================
from CDK_pywrapper import CDK


# q-RASAR without Weight Optimization

In [21]:
# ===============================================================
# ===================== Load QSAR Test Set =====================
# ===============================================================
qsar_test_path = r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\Test_set_Dermal_with_fingerprints_RDKit_CDK.xlsx"
qsar_df = pd.read_excel(qsar_test_path)
y_true = qsar_df['Outcome'].astype(int).values

# Models QSAR per fingerprint
models_per_fp = {
    'Morgan': r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\QSAR Model\Dermal_xgb_morgan.pkl",
    'MACCS': r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\QSAR Model\Dermal_SVM_MACCS.pkl",
    'APF':   r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\QSAR Model\Dermal_rf_apf.pkl",
    'RDKitCDK': r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\QSAR Model\Dermal_xgb_rdkitcdk.pkl"
}
loaded_models = {fp: joblib.load(path) for fp, path in models_per_fp.items()}

# Helper convert descriptor string
def convert_to_array(series):
    return np.array(series.apply(ast.literal_eval).tolist())

# Hitung probabilitas QSAR per fingerprint
probs_store = {}
for fp, model in loaded_models.items():
    if fp == 'RDKitCDK':
        X_input = qsar_df.drop(columns=['SMILES', 'Morgan_Descriptors', 'MACCS_Descriptors', 'APF_Descriptors', 'RDK_Descriptors', 'Outcome']).values
    else:
        X_input = convert_to_array(qsar_df[f"{fp}_Descriptors"])
    probs_store[fp] = model.predict_proba(X_input)[:,1]

# QSAR consensus probabilitas
qsar_consensus_probs = np.mean(list(probs_store.values()), axis=0)
qsar_df['QSAR_Consensus_Prob'] = qsar_consensus_probs

# ===============================================================
# ===================== Read-Across =============================
# ===============================================================
ra_train_file = r"C:\Fauzan\Manuscripts QSAR-RA 2\Old Endpoints\Acute Dermal Toxicity (manual split)\Read Across\Train_set_Dermal_balanced_with_fingerprints.xlsx"

train_df = pd.read_excel(ra_train_file)
y_train = train_df['Outcome'].values.astype(int)
fingerprint_columns = ['Morgan_Descriptors', 'MACCS_Descriptors', 'APF_Descriptors', 'RDK_Descriptors']
selected_fps = ['Morgan_Descriptors', 'MACCS_Descriptors', 'APF_Descriptors', 'RDK_Descriptors']  # bisa disesuaikan

# Convert SMILES ke Mol
train_df['Mol'] = train_df['SMILES'].apply(lambda x: Chem.MolFromSmiles(str(x)))
qsar_df['Mol']  = qsar_df['SMILES'].apply(lambda x: Chem.MolFromSmiles(str(x)))

# Prepare fingerprints
def prepare_fingerprints(df, fingerprint_cols):
    fps_dict = {}
    for col in fingerprint_cols:
        fps_dict[col] = []
        for fp in df[col]:
            if isinstance(fp, str):
                fp_list = ast.literal_eval(fp)
            else:
                fp_list = fp
            n_bits = len(fp_list)
            bv = ExplicitBitVect(n_bits)
            for i, bit in enumerate(fp_list):
                if int(bit):
                    bv.SetBit(i)
            fps_dict[col].append(bv)
    return fps_dict

train_fps = prepare_fingerprints(train_df, fingerprint_columns)
test_fps  = prepare_fingerprints(qsar_df, fingerprint_columns)

# Tanimoto similarity & predict RA
def compute_tanimoto_similarity(target_fp, reference_fps):
    return np.array([TanimotoSimilarity(target_fp, ref_fp) for ref_fp in reference_fps])

def predict_properties(test_fps_list, train_fps_list, y_train, k=5):
    predicted_probabilities = []
    for test_fp in test_fps_list:
        sims = compute_tanimoto_similarity(test_fp, train_fps_list)
        most_similar_indices = np.argsort(sims)[-k:]
        similar_outcomes = y_train[most_similar_indices]
        predicted_probabilities.append(np.mean(similar_outcomes))
    return np.array(predicted_probabilities)

# Hitung probabilitas RA per fingerprint
ra_probs_list = []
for fp in selected_fps:
    probs = predict_properties(test_fps[fp], train_fps[fp], y_train)
    ra_probs_list.append(probs)

# Konsensus RA probabilitas
ra_consensus_probs = np.mean(ra_probs_list, axis=0)
qsar_df['RA_Prob'] = ra_consensus_probs

# ===============================================================
# ===================== q-RASAR =================================
# ===============================================================
qsar_df['qRASAR_Prob'] = (qsar_df['QSAR_Consensus_Prob'] + qsar_df['RA_Prob']) / 2
qsar_df['qRASAR_Label'] = (qsar_df['qRASAR_Prob'] >= 0.5).astype(int)

# Hitung metrics q-RASAR
def compute_metrics(y_true, probs, threshold=0.5):
    preds = (probs >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()
    auc = roc_auc_score(y_true, probs)
    acc = accuracy_score(y_true, preds)
    sen = tp / (tp + fn) if (tp + fn) > 0 else 0
    spe = tn / (tn + fp) if (tn + fp) > 0 else 0
    return {"AUC": auc, "Accuracy": acc, "Sensitivity": sen, "Specificity": spe,
            "TN": tn, "FP": fp, "FN": fn, "TP": tp}

metrics_qrasar = compute_metrics(y_true, qsar_df['qRASAR_Prob'])
print("\n--- q-RASAR Performance ---")
for k, v in metrics_qrasar.items():
    print(f"{k}: {v:.4f}")

# ===============================================================
# ===================== Simpan hasil ============================
# ===============================================================
# 1️⃣ Simpan prediksi per senyawa
output_path_pred = r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\Evaluation\qRASAR_Consensus_RA_Predictions.xlsx"
os.makedirs(os.path.dirname(output_path_pred), exist_ok=True)
qsar_df.to_excel(output_path_pred, index=False)
print("\nHasil prediksi q-RASAR tersimpan ke:", output_path_pred)

# 2️⃣ Simpan performa metrics q-RASAR
metrics_df = pd.DataFrame([metrics_qrasar])
output_path_metrics = r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\Evaluation\qRASAR_Consensus_RA_Performance.xlsx"
metrics_df.to_excel(output_path_metrics, index=False)
print("Metrics q-RASAR tersimpan ke:", output_path_metrics)


C:\Users\USER-PC\anaconda3\envs\my-work-env\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\USER-PC\anaconda3\envs\my-work-env\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\USER-PC\anaconda3\envs\my-work-env\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from versi


--- q-RASAR Performance ---
AUC: 0.9306
Accuracy: 0.8627
Sensitivity: 0.8289
Specificity: 0.8961
TN: 69.0000
FP: 8.0000
FN: 13.0000
TP: 63.0000

Hasil prediksi q-RASAR tersimpan ke: C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\Evaluation\qRASAR_Consensus_RA_Predictions.xlsx
Metrics q-RASAR tersimpan ke: C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\Evaluation\qRASAR_Consensus_RA_Performance.xlsx


# TARGETING TO HIGHEST AUC & ACC (Weight Optimzation)

In [3]:
# ===============================================================
# ===================== Load QSAR Test Set ======================
# ===============================================================
qsar_test_path = r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\Test_set_Dermal_with_fingerprints_RDKit_CDK.xlsx"
qsar_df = pd.read_excel(qsar_test_path)
y_true = qsar_df['Outcome'].astype(int).values

# Models QSAR per fingerprint
models_per_fp = {
    'Morgan': r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\QSAR Model\Dermal_xgb_morgan.pkl",
    'MACCS': r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\QSAR Model\Dermal_SVM_MACCS.pkl",
    'APF':   r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\QSAR Model\Dermal_rf_apf.pkl",
    'RDKitCDK': r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\QSAR Model\Dermal_xgb_rdkitcdk.pkl"
}
loaded_models = {fp: joblib.load(path) for fp, path in models_per_fp.items()}

def convert_to_array(series):
    return np.array(series.apply(ast.literal_eval).tolist())

# Hitung probabilitas QSAR per fingerprint
probs_store = {}
for fp, model in loaded_models.items():
    if fp == 'RDKitCDK':
        X_input = qsar_df.drop(columns=[
            'SMILES', 'Morgan_Descriptors', 'MACCS_Descriptors', 'APF_Descriptors',
            'RDK_Descriptors', 'Outcome'
        ]).values
    else:
        X_input = convert_to_array(qsar_df[f"{fp}_Descriptors"])
    probs_store[fp] = model.predict_proba(X_input)[:, 1]

qsar_consensus_probs = np.mean(list(probs_store.values()), axis=0)
qsar_df['QSAR_Consensus_Prob'] = qsar_consensus_probs

# ===============================================================
# ===================== Read-Across ==============================
# ===============================================================
ra_train_file = r"C:\Fauzan\Manuscripts QSAR-RA 2\Old Endpoints\Acute Dermal Toxicity (manual split)\Read Across\Train_set_Dermal_balanced_with_fingerprints.xlsx"

train_df = pd.read_excel(ra_train_file)
y_train = train_df['Outcome'].values.astype(int)

fingerprint_columns = ['Morgan_Descriptors', 'MACCS_Descriptors', 'RDK_Descriptors']
selected_fps = fingerprint_columns

train_df['Mol'] = train_df['SMILES'].apply(lambda x: Chem.MolFromSmiles(str(x)))
qsar_df['Mol'] = qsar_df['SMILES'].apply(lambda x: Chem.MolFromSmiles(str(x)))

def prepare_fingerprints(df, fingerprint_cols):
    fps_dict = {}
    for col in fingerprint_cols:
        fps_dict[col] = []
        for fp in df[col]:
            fp_list = ast.literal_eval(fp) if isinstance(fp, str) else fp
            n_bits = len(fp_list)
            bv = ExplicitBitVect(n_bits)
            for i, bit in enumerate(fp_list):
                if int(bit):
                    bv.SetBit(i)
            fps_dict[col].append(bv)
    return fps_dict

train_fps = prepare_fingerprints(train_df, fingerprint_columns)
test_fps  = prepare_fingerprints(qsar_df, fingerprint_columns)

def compute_tanimoto_similarity(target_fp, reference_fps):
    return np.array([TanimotoSimilarity(target_fp, ref_fp) for ref_fp in reference_fps])

def predict_properties(test_fps_list, train_fps_list, y_train, k=5):
    predicted_probabilities = []
    for test_fp in test_fps_list:
        sims = compute_tanimoto_similarity(test_fp, train_fps_list)
        idx = np.argsort(sims)[-k:]
        similar_outcomes = y_train[idx]
        predicted_probabilities.append(np.mean(similar_outcomes))
    return np.array(predicted_probabilities)

ra_probs_list = []
for fp in selected_fps:
    ra_probs_list.append(predict_properties(test_fps[fp], train_fps[fp], y_train))

ra_consensus_probs = np.mean(ra_probs_list, axis=0)
qsar_df['RA_Prob'] = ra_consensus_probs


# ===============================================================
# ============= BOOTSTRAP CI 95% (mean ± half-width) ============
# ===============================================================

def bootstrap_metrics(probs, y_true, n_bootstrap=1000, ci=0.95, random_state=42):
    rng = np.random.RandomState(random_state)
    preds = (probs >= 0.5).astype(int)

    acc_list, sen_list, spe_list, auc_list = [], [], [], []
    n_samples = len(y_true)

    for _ in range(n_bootstrap):
        idx = rng.choice(np.arange(n_samples), size=n_samples, replace=True)
        y_sample = y_true[idx]
        p_sample = probs[idx]
        pred_sample = preds[idx]

        tn, fp, fn, tp = confusion_matrix(y_sample, pred_sample).ravel()

        acc_list.append(accuracy_score(y_sample, pred_sample))
        sen_list.append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        spe_list.append(tn / (tn + fp) if (tn + fp) > 0 else 0)

        try:
            auc_list.append(roc_auc_score(y_sample, p_sample))
        except:
            auc_list.append(np.nan)

    lower = (1 - ci) / 2
    upper = 1 - lower

    def fmt(values):
        mean_val = np.nanmean(values)
        low = np.nanpercentile(values, 100 * lower)
        high = np.nanpercentile(values, 100 * upper)
        half_width = (high - low) / 2
        return f"{mean_val:.2f} ± {half_width:.2f}"

    return {
        "AUC_CI": fmt(auc_list),
        "ACC_CI": fmt(acc_list),
        "SEN_CI": fmt(sen_list),
        "SPE_CI": fmt(spe_list)
    }


# ===============================================================
# ================= q-RASAR Weight Sweep ========================
# ===============================================================

results = []
weights = np.arange(0.01, 1.00, 0.01)

for w in weights:
    qrasar_prob = w * qsar_consensus_probs + (1 - w) * ra_consensus_probs
    preds = (qrasar_prob >= 0.5).astype(int)

    tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()
    auc = roc_auc_score(y_true, qrasar_prob)
    acc = accuracy_score(y_true, preds)
    sen = tp / (tp + fn) if (tp + fn) > 0 else 0
    spe = tn / (tn + fp) if (tn + fp) > 0 else 0

    # 🎯 Bootstrap CI
    boot = bootstrap_metrics(qrasar_prob, y_true)

    results.append({
        'QSAR_Weight': round(w, 2),
        'RA_Weight': round(1-w, 2),

        'AUC': auc,
        'AUC_CI': boot["AUC_CI"],

        'Accuracy': acc,
        'ACC_CI': boot["ACC_CI"],

        'Sensitivity': sen,
        'SEN_CI': boot["SEN_CI"],

        'Specificity': spe,
        'SPE_CI': boot["SPE_CI"],

        'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp
    })

# Ranking
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="AUC", ascending=False)

# ===============================================================
# ===================== Save Output =============================
# ===============================================================
output_path = r"C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\Evaluation\qRASAR_WeightSweep_Performance.xlsx"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
results_df.to_excel(output_path, index=False)

print("Hasil selesai dan tersimpan ke:", output_path)
print("\nTop 5 kombinasi berdasarkan AUC:")
print(results_df.head(5))


C:\Users\USER-PC\anaconda3\envs\my-work-env\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\USER-PC\anaconda3\envs\my-work-env\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\USER-PC\anaconda3\envs\my-work-env\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from versi

Hasil selesai dan tersimpan ke: C:\Fauzan\Manuscripts QSAR-RA 2\q-RASAR\Acute Dermal Toxicity\Evaluation\qRASAR_WeightSweep_Performance.xlsx

Top 5 kombinasi berdasarkan AUC:
    QSAR_Weight  RA_Weight       AUC       AUC_CI  Accuracy       ACC_CI  \
98         0.99       0.01  0.945660  0.95 ± 0.03  0.849673  0.85 ± 0.05   
97         0.98       0.02  0.945489  0.94 ± 0.03  0.849673  0.85 ± 0.05   
96         0.97       0.03  0.945489  0.94 ± 0.03  0.849673  0.85 ± 0.05   
90         0.91       0.09  0.945318  0.94 ± 0.04  0.849673  0.85 ± 0.05   
89         0.90       0.10  0.945147  0.94 ± 0.04  0.849673  0.85 ± 0.05   

    Sensitivity       SEN_CI  Specificity       SPE_CI  TN  FP  FN  TP  
98     0.828947  0.83 ± 0.08      0.87013  0.87 ± 0.07  67  10  13  63  
97     0.828947  0.83 ± 0.08      0.87013  0.87 ± 0.07  67  10  13  63  
96     0.828947  0.83 ± 0.08      0.87013  0.87 ± 0.07  67  10  13  63  
90     0.828947  0.83 ± 0.08      0.87013  0.87 ± 0.07  67  10  13  63  
89 